# 🧠 OpenAI Vector Store Setup
Dieses Notebook lädt eine Datei hoch, erstellt einen Vector Store, verbindet ihn mit einem Assistant und ermöglicht die semantische Suche mit GPT-4.

In [13]:
# 📦 Installiere notwendige Pakete
!pip install openai python-dotenv


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [14]:
# 🔐 API-Key einbinden (über .env Datei empfohlen)
from dotenv import load_dotenv
import os
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [15]:
# 📁 Datei hochladen
from pathlib import Path

file_path = "../data/Hundeerziehung.docx"
file = openai.files.create(file=open(file_path, "rb"), purpose="assistants")
print("✅ Datei hochgeladen:", file.id)

✅ Datei hochgeladen: file-QDBXKxXsfMg1XmjH231Aht


In [16]:
# 🧠 Vector Store erstellen
from openai import OpenAI
client = OpenAI()
vector_store = client.vector_stores.create(name="MeinVektorStore")
print("📌 Vector Store ID:", vector_store.id)

📌 Vector Store ID: vs_67f9782d630c8191871a49b7c41cd132


In [17]:
# ➕ Datei zum Vector Store hinzufügen

vector_store_file = client.vector_stores.files.create(
  vector_store_id=vector_store.id,
  file_id=file.id
)

print("✅ Datei im Vector Store gespeichert")

✅ Datei im Vector Store gespeichert


In [18]:
# 👉 Liste aller Dateien im Vector Store abrufen
file_list = client.vector_stores.files.list(
    vector_store_id=vector_store.id
)

print("📁 Dateien im Vector Store:")
for f in file_list.data:
    file_info = client.files.retrieve(f.id)
    print(f"• {file_info.filename} (ID: {file_info.id})")

📁 Dateien im Vector Store:
• Hundeerziehung.docx (ID: file-QDBXKxXsfMg1XmjH231Aht)


In [19]:
# 🤖 Assistant erstellen
assistant = client.beta.assistants.create(
    name="Hunde-KI",
    instructions="Beantworte die Fragen ausschließlich mit den Inhalten aus dem Dokument. Wenn Du darin keine passende Antwort findest, gebe einen Hinweis aus, dass Du nichts gefunden hast.",
    tools=[{"type": "file_search"}],
    tool_resources={
        "file_search": {
            "vector_store_ids": ["vs_67f96f8dfd348191b3f2952ed650e3b0"]
        }
    },
    model="gpt-4o",
)
print("🧠 Assistant-ID:", assistant.id)

🧠 Assistant-ID: asst_p90hmgIqZ56e17TkFNjzqBvc


In [20]:
# 💬 Thread starten
thread = openai.beta.threads.create()
print("💬 Thread-ID:", thread.id)

💬 Thread-ID: thread_qAzSgnjGjM2iSNsuPt1AtkqK


In [21]:
# ❓ Frage stellen und Antwort erhalten
def frage_stellen(frage: str):
    openai.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=frage,
    )
    run = openai.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )
    messages = openai.beta.threads.messages.list(thread_id=thread.id)
    for msg in reversed(messages.data):
        if msg.role == "assistant":
            return msg.content[0].text.value
    return "Keine Antwort gefunden."

# Beispiel-Frage:
frage_stellen("Warum bellt ein Hund?")


'Ein Hund bellt oft aus einem instinktiven Bedürfnis heraus, auf seine Umgebung zu reagieren. Zum Beispiel kann ein Hund bellen, wenn jemand an der Tür klingelt, um vor einer vermeintlichen Gefahr zu warnen. Dieses Verhalten entspringt dem Territorialinstinkt und dem sozialen Rudelinstinkt. Hunde können auch bellen und jaulen, wenn sie alleine gelassen werden, weil sie glauben, dass sie für die Sicherheit ihrer Besitzer verantwortlich sind und sich Sorgen machen, dass die Besitzer nicht gut auf sich aufpassen【4:0†Hundeerziehung.docx】.'

👉 Jetzt kannst du oben deine Datei anpassen und beliebige Fragen stellen.